In [4]:
%pip install polars

Note: you may need to restart the kernel to use updated packages.


In [5]:
import polars as pl
from datetime import datetime

In [6]:
df = pl.DataFrame(
    {
        "integer" : [1, 2, 3],
        "date": [
            datetime(2025, 1, 1),
            datetime(2025, 1, 2),
            datetime(2025, 1, 3),
        ],
        "float": [4.0, 5.0, 6.0],
        "string": ["a", "b", "c"]
    }
)

print(df)

shape: (3, 4)
┌─────────┬─────────────────────┬───────┬────────┐
│ integer ┆ date                ┆ float ┆ string │
│ ---     ┆ ---                 ┆ ---   ┆ ---    │
│ i64     ┆ datetime[μs]        ┆ f64   ┆ str    │
╞═════════╪═════════════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01 00:00:00 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02 00:00:00 ┆ 5.0   ┆ b      │
│ 3       ┆ 2025-01-03 00:00:00 ┆ 6.0   ┆ c      │
└─────────┴─────────────────────┴───────┴────────┘


In [7]:
df.write_csv("output.csv")

In [8]:
df_csv = pl.read_csv("output.csv")
print(df_csv)

shape: (3, 4)
┌─────────┬────────────────────────────┬───────┬────────┐
│ integer ┆ date                       ┆ float ┆ string │
│ ---     ┆ ---                        ┆ ---   ┆ ---    │
│ i64     ┆ str                        ┆ f64   ┆ str    │
╞═════════╪════════════════════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01T00:00:00.000000 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02T00:00:00.000000 ┆ 5.0   ┆ b      │
│ 3       ┆ 2025-01-03T00:00:00.000000 ┆ 6.0   ┆ c      │
└─────────┴────────────────────────────┴───────┴────────┘


In [9]:
df.select(pl.col("date", 'float'))

date,float
datetime[μs],f64
2025-01-01 00:00:00,4.0
2025-01-02 00:00:00,5.0
2025-01-03 00:00:00,6.0


In [10]:
df_filter = df.filter(
    pl.col("date").is_between(datetime(2025, 1, 1), datetime(2025, 1, 2))
)
print(df_filter)

shape: (2, 4)
┌─────────┬─────────────────────┬───────┬────────┐
│ integer ┆ date                ┆ float ┆ string │
│ ---     ┆ ---                 ┆ ---   ┆ ---    │
│ i64     ┆ datetime[μs]        ┆ f64   ┆ str    │
╞═════════╪═════════════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01 00:00:00 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02 00:00:00 ┆ 5.0   ┆ b      │
└─────────┴─────────────────────┴───────┴────────┘


In [11]:
df_add_col = df.with_columns(pl.col("float").sum().alias("sum"), pl.col("date").min().alias("min"))
print(df_add_col)

shape: (3, 6)
┌─────────┬─────────────────────┬───────┬────────┬──────┬─────────────────────┐
│ integer ┆ date                ┆ float ┆ string ┆ sum  ┆ min                 │
│ ---     ┆ ---                 ┆ ---   ┆ ---    ┆ ---  ┆ ---                 │
│ i64     ┆ datetime[μs]        ┆ f64   ┆ str    ┆ f64  ┆ datetime[μs]        │
╞═════════╪═════════════════════╪═══════╪════════╪══════╪═════════════════════╡
│ 1       ┆ 2025-01-01 00:00:00 ┆ 4.0   ┆ a      ┆ 15.0 ┆ 2025-01-01 00:00:00 │
│ 2       ┆ 2025-01-02 00:00:00 ┆ 5.0   ┆ b      ┆ 15.0 ┆ 2025-01-01 00:00:00 │
│ 3       ┆ 2025-01-03 00:00:00 ┆ 6.0   ┆ c      ┆ 15.0 ┆ 2025-01-01 00:00:00 │
└─────────┴─────────────────────┴───────┴────────┴──────┴─────────────────────┘


In [12]:
df2 = pl.DataFrame(
    {
        "x": range(8),
        "y": ["A", "A", "A", "B", "B", "C", "X", "X"],
    }
)

print(df2)

shape: (8, 2)
┌─────┬─────┐
│ x   ┆ y   │
│ --- ┆ --- │
│ i64 ┆ str │
╞═════╪═════╡
│ 0   ┆ A   │
│ 1   ┆ A   │
│ 2   ┆ A   │
│ 3   ┆ B   │
│ 4   ┆ B   │
│ 5   ┆ C   │
│ 6   ┆ X   │
│ 7   ┆ X   │
└─────┴─────┘


In [13]:
df2_groupby = df2.group_by("y", maintain_order=True).len()
print(df2_groupby)

shape: (4, 2)
┌─────┬─────┐
│ y   ┆ len │
│ --- ┆ --- │
│ str ┆ u32 │
╞═════╪═════╡
│ A   ┆ 3   │
│ B   ┆ 2   │
│ C   ┆ 1   │
│ X   ┆ 2   │
└─────┴─────┘


In [15]:
# %pip install polars-u64-idx polars-lts-cpu
# %pip install 'polars[numpy, fsspec]'

In [21]:
chemin = 'C:/Users/kkodjo/Documents/OneDrive - INAM/Documents/ImpactActs/'

In [20]:
df_actes = pl.read_csv(chemin + 'Actes.csv', separator=";", encoding='utf8')

FileNotFoundError: No such file or directory (os error 2): C:/Users/kkodjo/Documents/OneDrive - INAM/Documents/ImpactActs/Actes.csv